In [1]:
#-*- encoding: iso-8859-15 -*-
import numpy as np
from easydict import EasyDict as edict

import torch
from torch.utils.data import Dataset, DataLoader
from torch.autograd import Variable
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

#### Configuration / parameters to set

In [2]:
args = edict()
args.seq_len = 30
args.offset = 4
args.cuda = False
args.batch_size = 1
args.num_layers = 3
args.hidden_size = 128

### Data Processing functions and classes

In [12]:
def prepare_text(textsource):
    text = ''
    with open(textsource) as txtsource:
        for line in txtsource:
            line = line.strip().lower()
            line = line.replace(',', '').replace('.', '')
            line = line.replace('»', '').replace('«', '')
            line = line.replace('"', '')
            text += ' ' + line
    text = text[:500] #### nachher wieder rauslöschen!!!
    return text
# Chevrons müssen noch weg

In [7]:
def prepare_data(text, seq_len, offset):
    # Get all the unique characters appearing in the text 
    chars = sorted(list(set(text)))
    char_idx = dict((c, i) for i, c in enumerate(chars))
    idx_char = dict((i, c) for i, c in enumerate(chars)) #### das brauchen wir später!!!
    no_classes = len(chars) # the nr. of unique characters corresponds to the nr. of classes
    
    # Define training samples by splitting the text
    sentences = []
    next_chars = []
    for i in range(0, len(text) - seq_len, offset):
        sentences.append(text[i: i + seq_len])
        next_chars.append(text[i + seq_len])
    print('nr training samples', len(sentences))
    
    # Generate features and labels using one-hot encoding
    X = np.zeros((len(sentences), seq_len, len(chars)), dtype='f')
    y = np.zeros((len(sentences)))
    
    for i, sentence in enumerate(sentences):
        for j, char in enumerate(sentence):
            X[i, j, char_idx[char]] = 1
        y[i] = char_idx[next_chars[i]]
        
    return X, y, no_classes

In [8]:
class TextDataset(Dataset):
    ''' A text dataset class which implements the abstract class torch.utils.data.Dataset. '''
    def __init__(self, text, seq_len, offset):
        self.data, self.target, self.no_classes = prepare_data(text, seq_len, offset)
        
    def __getitem__(self, index):
        ''' Get the data for one training sample (by index) '''
        return self.data[index,:,:], self.target[index] 
    
    def __len__(self):
        ''' Get the number of training samples '''
        return self.data.shape[0]

### LSTM functions and classes

In [7]:
class LSTM_RNN(nn.Module):
    
    def __init__(self, input_shape):
        super(LSTM_RNN, self).__init__()
        
        self.lstm = nn.LSTM(input_size = input_shape[0]*input_shape[1], hidden_size = args.hidden_size)
        self.linear = nn.Linear(in_features = args.hidden_size, out_features = input_shape[1])
        self.softmax = nn.Softmax()
        
        # LSTM needs hidden variable which is initialized in self.init_hidden(self)
        self.hidden = self.init_hidden()
    
    def init_hidden(self):
        h0 = Variable(torch.zeros(args.num_layers, args.batch_size, args.hidden_size))
        c0 = Variable(torch.zeros(args.num_layers, args.batch_size, args.hidden_size))
        return (h0, c0)
    
    def forward(self, x):
        lstm_out, self.hidden = self.lstm(x, self.hidden) # (h0, c0 are set to default values)
        res = self.softmax(self.linear(lstm_out[-1])) # use only the output of the last layer of lstm
        return res

In [8]:
# Training loop (one epoch)
def train(model, epoch):
    model.train()
    criterion = nn.CrossEntropyLoss() # use the cross-entropy loss
    total_loss = 0.0 # compute total loss over one epoch
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data = data.view(data.size(0), -1)
        if args.cuda:
            data, target = data.cuda(), target.cuda()   
        data, target = Variable(data), Variable(target)
        
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target.long()) # check how far away the output is from the original data
        loss.backward(retain_graph=True)
        optimizer.step()
        
        total_loss += loss.data[0]
    print('Total loss over epoch %s: %s' %(epoch, total_loss/len(train_loader.dataset)))

### Main code

In [13]:
# test the functions defined above
textsource = './Brown_Leseprobe.txt'
text = prepare_text(textsource)
feat, true_pred, no_classes = prepare_data(text, args.seq_len, args.offset)
print(feat.shape)
print(no_classes)
print(text)

('nr training samples', 493)
(493, 30, 36)
36
 ﻿während die historische zahnradbahn sich mühsam ihren weg den schwindelerregend steilen hang hinaufkrallte blickte edmond kirsch auf die gezackten bergspitzen hoch über ihm in der ferne hineingebaut in die flanke einer senkrecht aufragenden klippe schien das weitläufige klostergebäude über dem abgrund zu schweben als hätte es sich auf magische weise von der felswand gelöst dieser zeitlose zufluchtsort hatte die glühende sonne kataloniens den rauen wind in den bergen und andere unbilden des wetters und der geschichte nun schon seit mehr als vierhundert jahren überdauert ohne je von seiner ursprünglichen bestimmung abzukommen seine bewohner vor der modernen welt abzuschotten und ausgerechnet sie sind die ersten die nun die wahrheit erfahren dachte kirsch was für eine ironie er fragte sich wie sie reagieren würden schließlich waren die gefährlichsten männer auf erden immer und zu allen zeiten männer des glaubens gewesen – insbesondere wenn i

In [10]:
# Generate train and test loader from our data
train_text = prepare_text('./Brown_Leseprobe.txt')
train_set = TextDataset(train_text, args.seq_len, args.offset)
train_loader = DataLoader(train_set, batch_size = args.batch_size, shuffle=True)
#dataiter = iter(train_loader)
#print(dataiter.next())

# set further parameters
no_classes = train_set.no_classes
input_shape = (args.seq_len, no_classes) # seq_len * nr. of unique characters 

('nr training samples', 118)


In [11]:
# Generate model
rnn = LSTM_RNN(input_shape)
if args.cuda:
    rnn.cuda()
print(rnn)

LSTM_RNN(
  (lstm): LSTM(900, 128)
  (linear): Linear(in_features=128, out_features=30)
  (softmax): Softmax()
)


In [12]:
# Initialize the optimization algorithm
optimizer = optim.RMSprop(rnn.parameters(), lr=0.01)

In [ ]:
for epoch in range(2):
    train(rnn, epoch)

/home/immd-user/miniconda/lib/python2.7/site-packages/ipykernel_launcher.py:20: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


In [227]:
m = nn.Linear(20, 30)
input = autograd.Variable(torch.randn(128, 20))
print(input)
print(m(input))

Variable containing:
-2.0177 -0.3889 -0.7212  ...   0.4350 -0.6106  0.1054
-0.0112 -0.7952  0.1476  ...  -0.2159 -0.3143 -0.3222
 0.6312 -1.1751 -0.6134  ...  -0.9334  0.3747 -0.6186
          ...             ⋱             ...          
 0.0543 -1.7096  0.9420  ...  -1.6969 -0.2117 -0.0515
-1.7247 -0.9483  0.1662  ...   1.0048  0.8270  0.0178
-0.4684 -3.1775  0.4234  ...   0.5584 -0.3391 -1.6763
[torch.FloatTensor of size 128x20]

Variable containing:
 1.7183e-01  1.7316e-01  3.3180e-01  ...  -1.1094e+00 -3.6481e-01 -3.2444e-02
-6.2500e-01  1.0468e+00  2.1427e-01  ...   5.8637e-02 -6.9745e-02 -6.1828e-01
-5.1829e-02 -5.7512e-01 -1.4507e-01  ...   8.2479e-01 -2.2266e-01  2.5010e-01
                ...                   ⋱                   ...                
 7.1955e-01  3.1914e-01  3.4348e-01  ...  -1.0117e+00  6.3292e-02  2.9535e-01
 6.2800e-01  3.9284e-02 -1.0128e-01  ...  -6.2670e-01  1.1004e+00 -2.3716e-01
 1.1138e+00  1.7880e-01 -1.1428e-01  ...   4.8020e-01 -2.8164e-01 -3.1969e-0

In [251]:
rnn = nn.LSTM(10, 20, 2)
input = Variable(torch.randn(5, 3, 10))
h0 = Variable(torch.randn(2, 3, 20))
c0 = Variable(torch.randn(2, 3, 20))
output = rnn(input, (h0, c0))
print(type(output))

<type 'tuple'>


In [69]:
output = Variable(torch.rand(1,10))
target = Variable(torch.LongTensor([1]))
print(output, target)
criterion = nn.CrossEntropyLoss()
loss = criterion(output, target)
print(loss)

(Variable containing:
 0.8576  0.8065  0.5420  0.6971  0.9650  0.9788  0.8399  0.2232  0.8018  0.6732
[torch.FloatTensor of size 1x10]
, Variable containing:
 1
[torch.LongTensor of size 1]
)
Variable containing:
 2.2554
[torch.FloatTensor of size 1]

